[View in Colaboratory](https://colab.research.google.com/github/ktaletsk/CPF/blob/master/2D_Example/CPF_2D_toy.ipynb)

In [0]:
#@title ##Initialize notebook (run this cell, CTRL+SHIFT)
from sympy import *
from sympy.utilities.codegen import codegen
import numpy as np
from IPython.display import Markdown
import matplotlib.pyplot as plt
# import scipy.integrate as integrate
from IPython.display import HTML, Math
from sympy.utilities.lambdify import lambdify

def latex_print(expr):
  display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/" "latest.js?config=default'></script>"))
  return Math(latex(expr))

#Number of dimensions
n=2

We are working with 2-dimensional system with free energy in a form:
\begin{equation}
\frac{F(x,y)}{k_B T} = 4(1-x)^2 + 4 \left(y - \frac{\sin a(x-c)}{b}\right)^2
\end{equation}
And friction in the detailed model is the same everywhere:
\begin{equation}
\frac{\zeta(x,y)}{k_B T} = 1
\end{equation}

You can choose parameters for this system below by draging sliders for $a$, $b$ and $c$

In [0]:
#@title Choose frequency, amplitude and phase of walls { display-mode: "form", run: "auto" }
a = 12 #@param {type:"slider", default:8, min:1, max:12, step:0.1}
b = 1.71 #@param {type:"slider", default:0.1, min:0.01, max:2, step:0.1}
c = -2.2 #@param {type:"slider", default:0, min:-5, max:5, step:0.1}

A = [a]
B = [b]
C = [c]
H = [4, 4]

#Define variables
x = symbols('x0:%d'%n)

def floor(x,A,B,C):
    return sin(A*(x-C))/B

def wall(x,y,A,B,C):
    return (y - floor(x,A,B,C))**2

def free_energy(n, x):
    result = H[0]*(1-x[0])**2
    for i in range(1, n):
        result += H[i] * wall(x[0], x[i], A[i-1], B[i-1], C[i-1])
    return result

def free_energy_CG(x):
    return H[0]*(1-x[0])**2

def zeta_inv(n, x):
    return 1 #Constant friction

def tshift(n,x,i):
    return diff(zeta_inv(n,x), x[i])-zeta_inv(n, x)*diff(free_energy(n,x), x[i])

def wshift(n,x):
    return sqrt(2.0*zeta_inv(n, x))


func = lambdify(x, free_energy(n, x),'numpy') # returns a numpy-ready function
Xplot = np.arange(0,2,0.01)
Yplot = np.arange(-5,5,0.01)
X, Y = np.meshgrid(Xplot, Yplot)
F = func(X, Y)
fig = plt.figure(figsize=(10, 8))
ax1 = fig.add_subplot(111)
CS3 = plt.contourf(X, Y, F, 100, cmap=plt.cm.hot)
fig.colorbar(CS3, ax=ax1)
ax1.set_title(r'$\frac{F}{k_B T}$')
ax1.set_xlabel(r'$x$')
ax1.set_ylabel(r'$y$')
for item in ([ax1.title, ax1.xaxis.label, ax1.yaxis.label] +
             ax1.get_xticklabels() + ax1.get_yticklabels()):
    item.set_fontsize(20)
plt.show()
latex_print(r'\frac{F}{k_B T} =' + latex(free_energy(n, x)))

You can see the free energy lanscape for detailed model from above, darker regions represent low free energy, bright regions represent high free energy and pose obstacles for diffusing Brownian particles